#### **Agentic RAG application using LangGraph**




In [ ]:
pip install langchain_community langchain langchain_huggingface langchain-core langgraph langchain_google_genai pypdf faiss-cpu

In [ ]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [19]:
import os
from typing import TypedDict, List, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document


# Configuration
DEFAULT_PDF_DIRECTORY = "/content/data"
FAISS_INDEX_PATH = "./faiss_index"


# Enhanced State Definition
class AgentState(TypedDict):
    messages: list
    mode: str
    pdf_directory: str
    response: str
    source_documents: List[Document]


# Initialize LLM with streaming enabled
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    api_key=api_key,
    #max_output_tokens= 1000,
    streaming=True
)


# Initialize embeddings globally
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


# Node 1: Direct LLM
def direct_llm_node(state: AgentState):
    """Direct interaction with LLM with conversation history"""
    messages = state["messages"]
    response = llm.invoke(messages)

    return {
        "response": response.content,
        "messages": messages + [AIMessage(content=response.content)],
        "source_documents": []
    }


# Node 2: RAG Pipeline with conversation history
def rag_pipeline_node(state: AgentState):
    """Complete RAG pipeline with conversation history support"""
    messages = state["messages"]
    current_query = messages[-1].content

    pdf_directory = state.get("pdf_directory", DEFAULT_PDF_DIRECTORY) or DEFAULT_PDF_DIRECTORY

    # Load or create FAISS index
    if os.path.exists(FAISS_INDEX_PATH):
        print("Loading existing FAISS index...")
        vectorstore = FAISS.load_local(
            FAISS_INDEX_PATH,
            embeddings,
            allow_dangerous_deserialization=True
        )
    else:
        print(f"Creating new FAISS index from directory: {pdf_directory}")

        # Load all PDFs from directory
        loader = PyPDFDirectoryLoader(
            path=pdf_directory,
            glob="**/*.pdf",  # Load all PDFs recursively
            recursive=True,    # Search subdirectories
            silent_errors=False  # Show errors if any
        )

        documents = loader.load()
        print(f" Loaded {len(documents)} pages from PDF files in {pdf_directory}")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=150
        )
        chunks = text_splitter.split_documents(documents)
        print(f" Created {len(chunks)} chunks from documents")

        vectorstore = FAISS.from_documents(chunks, embeddings)
        vectorstore.save_local(FAISS_INDEX_PATH)
        print(f" FAISS index saved to {FAISS_INDEX_PATH}")

    # Retrieval
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    relevant_docs = retriever.invoke(current_query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Build conversation history for context
    conversation_history = []
    for msg in messages[:-1]:  # Exclude current query
        if isinstance(msg, HumanMessage):
            conversation_history.append(f"User: {msg.content}")
        elif isinstance(msg, AIMessage):
            conversation_history.append(f"Assistant: {msg.content}")

    history_text = "\n".join(conversation_history) if conversation_history else "No previous conversation."

    # Enhanced prompt with conversation history
    prompt = f"""Based on the conversation history and context below, answer the current question.

Conversation History:
{history_text}

Context from documents:
{context}

Current Question: {current_query}

Answer (be specific and helpful):"""

    response = llm.invoke(prompt)

    return {
        "response": response.content,
        "messages": messages + [AIMessage(content=response.content)],
        "source_documents": relevant_docs
    }


# Routing Function
def route_query(state: AgentState) -> Literal["direct_llm", "rag_pipeline"]:
    """Route based on mode"""
    if state["mode"] == "rag":
        return "rag_pipeline"
    return "direct_llm"


# Build Graph
workflow = StateGraph(AgentState)
workflow.add_node("direct_llm", direct_llm_node)
workflow.add_node("rag_pipeline", rag_pipeline_node)

workflow.add_conditional_edges(
    START,
    route_query,
    {
        "direct_llm": "direct_llm",
        "rag_pipeline": "rag_pipeline"
    }
)

workflow.add_edge("direct_llm", END)
workflow.add_edge("rag_pipeline", END)

# Compile
app = workflow.compile()


#  CHAT FUNCTION
conversation_history = []


def chat(query: str, mode: str = "rag", pdf_directory: str = ""):
    """
    Chat function that maintains conversation history

    Args:
        query: User's question
        mode: "rag" or "direct"
        pdf_directory: Path to directory containing PDFs (empty for default)

    Returns:
        dict with response and source_documents
    """
    # Add user message to history
    conversation_history.append(HumanMessage(content=query))

    # Invoke with full conversation history
    result = app.invoke({
        "messages": conversation_history,
        "mode": mode,
        "pdf_directory": pdf_directory,
        "response": "",
        "source_documents": []
    })

    # Add AI response to history
    conversation_history.append(AIMessage(content=result["response"]))

    return result


def reset_conversation():
    """Clear conversation history to start fresh"""
    global conversation_history
    conversation_history = []
    print(" Conversation history cleared!")


def reset_faiss_index():
    """Delete FAISS index to force recreation from PDFs"""
    import shutil
    if os.path.exists(FAISS_INDEX_PATH):
        shutil.rmtree(FAISS_INDEX_PATH)
        print(f" FAISS index deleted from {FAISS_INDEX_PATH}")
    else:
        print(" No FAISS index found to delete")


In [20]:
# Example 1 - Direct LLM
print("=" * 50)
print("Example 1: Direct LLM Mode")
print("=" * 50)

result0 = chat("What is Generative AI", mode="direct")

print("\n🤖 Response: ", end="")
for char in result0["response"]:
    print(char, end="", flush=True)
print("\n")

Example 1: Direct LLM Mode

🤖 Response: Of course! Here is a comprehensive explanation of Generative AI, broken down for easy understanding.

### The Simple Analogy: The Creative Student

Imagine two types of students studying for a history test.

*   **The Memorizer (Traditional AI):** This student reads the textbook and memorizes all the facts and dates. If you ask, "When was the Battle of Hastings?" they can instantly answer "1066." They are excellent at classifying and predicting based on the exact data they've learned.

*   **The Creator (Generative AI):** This student also reads the textbook, but instead of just memorizing, they learn the *patterns*, the *causes and effects*, and the *storytelling style* of the historian. If you ask them to "Write a short, dramatic paragraph from the perspective of a soldier at the Battle of Hastings," they can create something entirely new that is consistent with the style and facts they learned.

**Generative AI is the creative student.** It do

In [21]:
# Example 2 - Direct LLM - follow-up question
print("=" * 50)
print("Example 2: Direct LLM Follow-up")
print("=" * 50)

result00 = chat("How it was different from traditional machine learning?", mode="direct")

print("\n Response: ", end="")
for char in result00["response"]:
    print(char, end="", flush=True)
print("\n")

Example 2: Direct LLM Follow-up

🤖 Response: Excellent question. This gets to the heart of the recent AI revolution. The difference lies in their fundamental **goal and output**.

Let's use a clear analogy to start.

### The Analogy: The Art Critic vs. The Artist

*   **Traditional Machine Learning is the Art Critic.** It can study thousands of paintings by Van Gogh. After its training, you can show it a new painting, and it can tell you with high accuracy, "Yes, that is a Van Gogh" or "No, that is a forgery." It can *classify*, *predict*, and *identify*. **It judges existing work.**

*   **Generative AI is the Artist.** It also studies thousands of paintings by Van Gogh. But instead of learning to judge, it learns the patterns, the brush strokes, the color palettes, and the essence of his style. Then, if you ask it to "Paint a picture of a lighthouse in the style of Van Gogh," it can create a **brand new, original painting** that has never existed before but looks authentically like h

In [22]:
# Example 3: First RAG query
print("=" * 50)
print("Example 3: RAG Mode (Multiple PDFs)")
print("=" * 50)

result1 = chat("What is Beam Search?", mode="rag")

print("\n Response: ", end="")
for char in result1["response"]:
    print(char, end="", flush=True)

print("\n\n--- Source Documents ---")
for idx, doc in enumerate(result1["source_documents"], 1):
    print(f"\n📄 Source {idx}:")
    print(f"   Page: {doc.metadata.get('page', 'N/A')}")
    print(f"   File: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:250]}...")

Example 3: RAG Mode (Multiple PDFs)
Creating new FAISS index from directory: /content/data
 Loaded 56 pages from PDF files in /content/data
 Created 110 chunks from documents
 FAISS index saved to ./faiss_index

🤖 Response: Of course. Based on the provided context and our previous conversations, here is a detailed explanation of Beam Search.

We'll start with an analogy, just like before.

### The Analogy: Navigating a Maze

Imagine you're in a maze and want to find the exit as quickly as possible. You have two strategies:

*   **Greedy Decoding (The Impulsive Explorer):** At every intersection, you instantly take the path that *looks* the most promising or points most directly toward the exit, without considering any other options. This is fast, but you might run into a dead end and have to backtrack, or you might miss a slightly longer but ultimately much faster route.

*   **Beam Search (The Cautious Team of Explorers):** At every intersection, you don't just choose one path. Instea

In [23]:
# Example 4: Follow-up question (uses conversation history!)
print("\n" + "=" * 50)
print("Example 4: Follow-up Question")
print("=" * 50)

result2 = chat("What was the last question I asked you?", mode="rag")

print("\n🤖 Response: ", end="")
for char in result2["response"]:
    print(char, end="", flush=True)
print("\n")

print("\n--- Source Documents ---")
for idx, doc in enumerate(result2["source_documents"], 1):
    print(f"\n📄 Source {idx}:")
    print(f"   Page: {doc.metadata.get('page', 'N/A')}")
    print(f"   File: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:250]}...")


Example 4: Follow-up Question
Loading existing FAISS index...

🤖 Response: Based on the conversation history, the last question you asked me was:

**"What is Beam Search?"**

In response, I explained it using an analogy of a "cautious team of explorers" navigating a maze, compared it to Greedy Decoding, and provided a step-by-step example of how it works to generate more coherent text.


--- Source Documents ---

📄 Source 1:
   Page: 0
   File: /content/data/50 LLM Interview Questions.pdf
   Content: TOPTOPTOP50 LLM50 LLM50 LLM
Bhavishya Pandit
Interview QuestionsInterview Questions...

📄 Source 2:
   Page: 38
   File: /content/data/50 LLM Interview Questions.pdf
   Content: Q36. What is Chain-of-Thought (CoT) prompting, and how does it
improve complex reasoning in LLMs?
-Chain-of-Thought (CoT) prompting helps LLMs handle complex
reasoning by encouraging them to break down tasks into smaller,
sequential steps. This impro...

📄 Source 3:
   Page: 10
   File: /content/data/50 LLM Interv

In [24]:
# Example 5: Follow-up question (uses conversation history!)
print("\n" + "=" * 50)
print("Example 5: Follow-up Question")
print("=" * 50)

result3 = chat("What was the last question I asked you?", mode="direct")

print("\n🤖 Response: ", end="")
for char in result3["response"]:
    print(char, end="", flush=True)
print("\n")

print("\n--- Source Documents ---")
for idx, doc in enumerate(result3["source_documents"], 1):
    print(f"\n📄 Source {idx}:")
    print(f"   Page: {doc.metadata.get('page', 'N/A')}")
    print(f"   File: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:250]}...")


Example 5: Follow-up Question

🤖 Response: The last question you asked me was:

**"What was the last question I asked you?"**

Before that, you asked me about "What is Beam Search?".


--- Source Documents ---
